# Unsupervised clustering of friends

In [2]:
%matplotlib inline

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import glob
import re
from functools import partial

import transliterate
from transliterate.contrib.languages.bg.translit_language_pack import BulgarianLanguagePack
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bulstem import stem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [4]:
ME = "Павел Богданов"

### Reading data

In [5]:
def json_to_df(filepath, friend_name=True):
    """
    Reads facebook messenger's JSON file and returns a pandas Dataframe.
    
    Doesn't return a dataframe if the participants are more
    than two people(no group chats).
    
    Works only on the "Messages" json files downloaded through
    Facebook's "Download Your Information" section.
    
    Parameters
    ----------
    filepath : string
        Filepath to the JSON file.
    friend_name : boolean, default True
        If True, adds an aditional column "friend_name" to the df.
    Returns
    -------
    result : Dataframe        
    """
    # Fixes bad encoding
    fix_mojibake_escapes = partial(re.compile(rb'\\u00([\da-f]{2})').sub, lambda m: bytes.fromhex(m.group(1).decode()))
    
    # Need to read as binary to decode correctly
    with open(filepath, 'rb') as file:    
        repaired = fix_mojibake_escapes(file.read())
        data = json.loads(repaired.decode('utf8'), strict=False)
        
        # No group chats!
        if len(data['participants']) == 2:
            result = pd.DataFrame.from_dict(data['messages'])
            
            # Additional column
            if friend_name:
                participants = pd.Series(data['participants']).apply(pd.Series)
                for name in participants.name:
                    if not name == ME:
                        result['friend_name'] = name
            return result

In [6]:
all_files = glob.glob("messages/inbox/*/message_1.json")
data = pd.concat((json_to_df(filename) for filename in all_files), ignore_index=True, sort=False)

### Processing data

In [7]:
data.timestamp_ms = pd.to_datetime(data.timestamp_ms, unit='ms')
data = data.sort_values('timestamp_ms')
data = data.drop_duplicates('timestamp_ms')
data = data.set_index('timestamp_ms', verify_integrity=True)
data.index.names = ['timestamp']
data.tail()

,content,sender_name,share,type,friend_name,photos,sticker,audio_files,gifs,reactions,videos,files,call_duration,missed,users,plan
timestamp,,,,,,,,,,,,,,,,
2019-07-22 15:09:15.928,"добре, чакай сега недей да пишеш за известно в...",Павел Богданов,NaN,Generic,Daniel Petrov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 15:09:31.911,some English text 123 wohoo yeaaah !!! xD,Павел Богданов,NaN,Generic,Daniel Petrov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 15:09:36.887,NaN,Daniel Petrov,NaN,Generic,Daniel Petrov,NaN,{'uri': 'messages/stickers_used/39178562_15051...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 15:09:55.550,sh1okavica bum bam nqkvi dumi,Павел Богданов,NaN,Generic,Daniel Petrov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 15:10:12.635,Данчо е супер пич 😎,Павел Богданов,NaN,Generic,Daniel Petrov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
mess = data[(data.type=='Generic') | (data.type=='Share')][['sender_name', 'content', 'friend_name', 'type', 'share', 'reactions']]
mess.content.dropna(inplace=True)
mess.sender_name = mess.sender_name.astype('category')
mess.friend_name = mess.friend_name.astype('category')
mess.type = mess.type.astype('category')
mess.content = mess.content.str.lower()
mess.content = mess.content.replace(np.nan, '0')
mess = mess[mess.content!='0']

In [13]:
mess.tail()

,sender_name,content,friend_name,type,share,reactions
timestamp,,,,,,
2019-07-22 15:08:48.294,Daniel Petrov,ако мн го закъсаш с събирането на съобщенията ...,Daniel Petrov,Generic,NaN,NaN
2019-07-22 15:09:15.928,Павел Богданов,"добре, чакай сега недей да пишеш за известно в...",Daniel Petrov,Generic,NaN,NaN
2019-07-22 15:09:31.911,Павел Богданов,some english text 123 wohoo yeaaah !!! xd,Daniel Petrov,Generic,NaN,NaN
2019-07-22 15:09:55.550,Павел Богданов,sh1okavica bum bam nqkvi dumi,Daniel Petrov,Generic,NaN,NaN
2019-07-22 15:10:12.635,Павел Богданов,данчо е супер пич 😎,Daniel Petrov,Generic,NaN,NaN


### Top friends

In [161]:
n = 15
space_mess = mess.copy()
space_mess.content = space_mess.content.str.split()
space_mess.content.dropna(inplace=True)
space_mess['wordcount_space'] = space_mess.content.apply(len)
best_friends = space_mess.groupby('sender_name').wordcount_space.sum().nlargest(len(list(space_mess.sender_name.unique())))
best_space_mess = space_mess[space_mess.friend_name.isin(list(best_friends[1:n+1].index))]
best_space_mess.reset_index(inplace=True)

### Transliteration 

In [149]:
BulgarianLanguagePack.mapping = ('abwvgdejziyklmnoprstufhc461q', 'абввгдежзийклмнопрстуфхцчшъя')
"""
my additions:
    "1": "ъ",
    "6": "ш",
    "4": "ч",
    "q": "я",
    "w": "в",
    "j": "ж",
    "c": "ц",
    
removed all the uppercase characters for improved performance
ABVWGDEZIYKLMNOPRSTUFHCQ:АБВВГДЕЗИЙКЛМНОПРСТУФХЦЯ    
"""


BulgarianLanguagePack.reversed_specific_mapping = ('ьъ', 'y1')


BulgarianLanguagePack.pre_processor_mapping = {
    'zh': 'ж',
    'ts': 'ц',
    'ch': 'ч',
    'sh': 'ш',
    'шt': 'щ', # fixed
    'yu': 'ю',
    'ya': 'я',
    
    # my additions:
    "6t": "щ",
    "1o": "ьо",    
    "1i": "ъй",
    "ai": "ай",
    # "йо" only after vowel
    # and beginning of word
    "yo": "йо",
    "bй": "бь",
    "vй": "вь",
    "wй": "вь",
    "gй": "гь",
    "dй": "дь",
    "jй": "жь",
    "жй": "жь",
    "zй": "зь",
    "kй": "кь",
    "lй": "ль",
    "mй": "мь",
    "nй": "нь",
    "pй": "пь",
    "rй": "рь",
    "sй": "сь",
    "tй": "ть",
    "fй": "фь",
    "cй": "ць",
    "чй": "чь",
    "шй": "шь",
    "щй": "щь"}
"""
    removed:
    'Zh': 'Ж',
    'Ts': 'Ц',
    'Ch': 'Ч',
    'Sh': 'Ш',
    'Шt': 'Щ', # fixed
    'Yu': 'Ю',
    'Ya': 'Я',
"""

# Instead of callng the language code each time
translit_bg = transliterate.get_translit_function('bg')

### Wordlists

In [152]:
en_wordlist = pd.read_csv('en_words.csv', header=None, names=['word'])
bg_wordlist = pd.read_csv('bg_full.txt', header=None, delim_whitespace=True, names=['word', 'frequency'])
bg_wordlist = bg_wordlist.dropna()
bg_wordlist = bg_wordlist[bg_wordlist.frequency > 2]
bg_wordlist = bg_wordlist[bg_wordlist.word.str.contains(pat=r'[a-zA-Z]+')==False].reset_index(drop=True)

### Stop words

In [150]:
bg_stopwords = pd.read_csv('https://raw.githubusercontent.com/Alir3z4/stop-words/master/bulgarian.txt', header=None, names=['word']).word
en_stopwords = pd.Series(stopwords.words('english'))
combined_stopwords = pd.concat([bg_stopwords, en_stopwords])

### Word split

In [162]:
def word_split(dataframe):
    rows = list()
    for row in dataframe[['sender_name', 'content', 'timestamp']].iterrows():
        r = row[1]
        for word in r.content:
            rows.append((r.sender_name, word, r.timestamp))

    return pd.DataFrame(rows, columns=['sender_name', 'word', 'timestamp'])

In [178]:
words_by_space = word_split(best_space_mess)

### Extracting valid words

In [179]:
words_by_space = words_by_space[words_by_space.word.apply(len) >= 3]
words_by_space.sender_name = words_by_space.sender_name.astype('category')
words_by_space.word = words_by_space.word.astype('category')

In [180]:
bg_words = words_by_space[(words_by_space.word.isin(bg_wordlist.word))
                              & (words_by_space.word.isin(en_wordlist.word)==False)]

en_words = words_by_space[(words_by_space.word.isin(en_wordlist.word)) 
                              & (words_by_space.word.isin(bg_wordlist.word)==False)]

recycled_words = words_by_space[(words_by_space.word.isin(bg_wordlist.word)==False) 
                                    & (words_by_space.word.isin(en_wordlist.word)==False)]

In [181]:
recycled_words = recycled_words.copy()
recycled_words.word = recycled_words.word.apply(translit_bg)
recycled_words = recycled_words[recycled_words.word.isin(bg_wordlist.word)]
bg_words = bg_words[bg_words.word.isin(bg_stopwords)==False]
en_words = en_words[en_words.word.isin(en_stopwords)==False]
recycled_words = recycled_words[recycled_words.word.isin(bg_stopwords)==False]
bg_words.word = bg_words.word.apply(stem)
en_words.word = en_words.word.apply(PorterStemmer().stem)
recycled_words.word = recycled_words.word.apply(stem)

In [182]:
processed_words = pd.concat(objs=[bg_words, en_words, recycled_words])

In [192]:
processed_mess = processed_words.groupby(["sender_name"]).word.agg(lambda x: ' '.join(x))

In [193]:
processed_mess = pd.DataFrame(processed_mess)
processed_mess = processed_mess.reset_index(level=0).set_index('sender_name')
processed_mess.columns = ['content']

### Model

In [213]:
vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.4, stop_words=combined_stopwords.tolist())
X = vectorizer.fit_transform(raw_documents=processed_mess.content)
X_norm = normalize(X)

In [214]:
true_k = n + 1
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=10, n_jobs=8)
model.fit(X_norm)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
    n_clusters=16, n_init=10, n_jobs=8, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [215]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [220]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind])

Cluster 0:
 доктор
 незна
 щот
 ади
 мале
 дет
 принцип
 даскал
 тоест
 йес
 дооб
 смята
 мдаа
 сиг
 еее
 мамк
 тамън
 китар
 епизод
 въобще
Cluster 1:
 like
 know
 принцип
 тоз
 hate
 някакв
 обичам
 fuck
 тия
 love
 shit
 one
 kind
 want
 say
 баси
 бургас
 доб
 бтв
 мрънка
Cluster 2:
 дае
 тря
 тей
 тия
 данн
 някъв
 ama
 амии
 dae
 ползва
 схем
 скив
 чек
 sum
 кви
 услови
 цикъл
 кода
 che
 лайн
Cluster 3:
 ama
 sum
 ako
 пук
 ima
 тряа
 che
 незна
 taka
 aha
 що
 ami
 tam
 imam
 тебе
 sled
 десе
 щот
 aid
 куде
Cluster 4:
 well
 like
 дае
 лол
 щот
 идк
 know
 тряа
 yeah
 тебе
 принцип
 get
 ний
 мене
 тея
 one
 баси
 няа
 въобще
 знайш
Cluster 5:
 ахам
 незна
 виктор
 смята
 щот
 сиг
 нощ
 бахт
 ako
 довечер
 събо
 тряя
 ami
 кола
 ехее
 даскал
 тоест
 ахаа
 дена
 къщи
Cluster 6:
 оке
 сиг
 ирин
 уоу
 даа
 урок
 идк
 sum
 фамили
 che
 пием
 уау
 вино
 готова
 тия
 рожд
 мед
 някакв
 кум
 звън
Cluster 7:
 баси
 найс
 авер
 дае
 ехе
 щот
 няа
 тряа
 haha
 тоест
 фак
 чек
 павк
 де